In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.mixed_precision import Policy, set_global_policy

policy = Policy('mixed_float16')
set_global_policy(policy)

INPUT_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

base_model = EfficientNetB0(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    global_average_layer,
    output_layer
])

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_checkpoint.keras', save_best_only=True)

print("Starting training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Fine-tuning the model...")
history_fine = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

model.save('The_model.keras')
print("Model training and fine-tuning complete!")


Found 5889 images belonging to 5 classes.
Found 1471 images belonging to 5 classes.
Starting training...
Epoch 1/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2168s 12s/step - accuracy: 0.2795 - loss: 1.5979 - val_accuracy: 0.5486 - val_loss: 1.2787 - learning_rate: 1.0000e-04
Epoch 2/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2470s 13s/step - accuracy: 0.6055 - loss: 1.2007 - val_accuracy: 0.6159 - val_loss: 1.1147 - learning_rate: 1.0000e-04
Epoch 3/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2303s 12s/step - accuracy: 0.6531 - loss: 1.0748 - val_accuracy: 0.6533 - val_loss: 1.0097 - learning_rate: 1.0000e-04
Epoch 4/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2240s 12s/step - accuracy: 0.6753 - loss: 0.9908 - val_accuracy: 0.6492 - val_loss: 0.9564 - learning_rate: 1.0000e-04
Epoch 5/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2142s 12s/step - accuracy: 0.6945 - loss: 0.9204 - val_accuracy: 0.6859 - val_loss: 0.8980 - learning_rate: 1.0000e-04
Epoch 6/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2143s 12s/step - accuracy: 0.7067 - loss: 0.8735 - val_accur

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# Set mixed precision policy for faster training
policy = Policy('mixed_float16')
set_global_policy(policy)

INPUT_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

# ImageDataGenerator setup with preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

# Load ResNet50 model with pretrained weights
base_model = ResNet50(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Add custom classification head
global_average_layer = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    global_average_layer,
    output_layer
])

# Compile the model with Adam optimizer
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks for efficient training
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_checkpoint_resnet.keras', save_best_only=True)

print("Starting training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

# Fine-tune the model
base_model.trainable = True
for layer in base_model.layers[:100]:  # Adjust layers for fine-tuning
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Fine-tuning the model...")
history_fine = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

# Save the trained model
model.save('The_model_resnet.keras')
print("Model training and fine-tuning complete!")


Found 5889 images belonging to 5 classes.
Found 1471 images belonging to 5 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
Starting training...
Epoch 1/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1903s 10s/step - accuracy: 0.2967 - loss: 1.5891 - val_accuracy: 0.5561 - val_loss: 1.1354 - learning_rate: 1.0000e-04
Epoch 2/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1782s 10s/step - accuracy: 0.6137 - loss: 1.0671 - val_accuracy: 0.6594 - val_loss: 0.9294 - learning_rate: 1.0000e-04
Epoch 3/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2043s 11s/step - accuracy: 0.6981 - loss: 0.8885 - val_accuracy: 0.7247 - val_loss: 0.8056 - learning_rate: 1.0000e-04
Epoch 4/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2389s 13s/step - accuracy: 0.7318 - loss: 0.8083 - val_accuracy: 0.7471 - val_loss: 0.7630 - learning_rate: 1.0000e-04
Epoch 5/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2505s 14s/step - accuracy: 0.7371 - loss: 0.7672 - val_accuracy: 0.7478 - val_loss: 0.7347 - learning_rate: 1.0000e-04
Epoch 6/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2550s 14

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.mixed_precision import Policy, set_global_policy

policy = Policy('mixed_float16')
set_global_policy(policy)

INPUT_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

base_model = ResNet50(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    global_average_layer,
    output_layer
])

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_checkpoint_resnet.keras', save_best_only=True)

print("Starting training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

if fine_tune:
    
    base_model.trainable = True
    for layer in base_model.layers[:150]:  
        layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("Fine-tuning the model...")
    history_fine = model.fit(
        train_generator,
        epochs=5,  
        validation_data=validation_generator,
        callbacks=[reduce_lr, early_stop, checkpoint]
    )

model.save('The_model_resnet.keras')
print("Model training complete!")


Found 5889 images belonging to 5 classes.
Found 1471 images belonging to 5 classes.
Starting training...


c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.3895 - loss: 1.5056

c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


185/185 ━━━━━━━━━━━━━━━━━━━━ 2255s 12s/step - accuracy: 0.3901 - loss: 1.5046 - val_accuracy: 0.5942 - val_loss: 1.0887 - learning_rate: 1.0000e-04
Epoch 2/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1888s 10s/step - accuracy: 0.6555 - loss: 1.0087 - val_accuracy: 0.6954 - val_loss: 0.8905 - learning_rate: 1.0000e-04
Epoch 3/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 2012s 11s/step - accuracy: 0.7135 - loss: 0.8688 - val_accuracy: 0.7186 - val_loss: 0.8065 - learning_rate: 1.0000e-04
Epoch 4/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1853s 10s/step - accuracy: 0.7424 - loss: 0.7882 - val_accuracy: 0.7335 - val_loss: 0.7493 - learning_rate: 1.0000e-04
Epoch 5/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1919s 10s/step - accuracy: 0.7565 - loss: 0.7184 - val_accuracy: 0.7702 - val_loss: 0.7103 - learning_rate: 1.0000e-04
Epoch 6/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1942s 10s/step - accuracy: 0.7665 - loss: 0.7093 - val_accuracy: 0.7927 - val_loss: 0.6582 - learning_rate: 1.0000e-04
Epoch 7/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1519s 8s/step - a

NameError: name 'fine_tune' is not defined

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.mixed_precision import Policy, set_global_policy

policy = Policy('mixed_float16')
set_global_policy(policy)

fine_tune = False  

INPUT_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

base_model = ResNet50(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    global_average_layer,
    output_layer
])

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_checkpoint_resnet.keras', save_best_only=True)

print("Starting training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

if fine_tune:
    base_model.trainable = True
    for layer in base_model.layers[:150]:  
        layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("Fine-tuning the model...")
    history_fine = model.fit(
        train_generator,
        epochs=5,  
        validation_data=validation_generator,
        callbacks=[reduce_lr, early_stop, checkpoint]
    )

model.save('The_model_resnet.keras')
print("Model training complete!")


Found 5889 images belonging to 5 classes.
Found 1471 images belonging to 5 classes.
Starting training...


c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3651 - loss: 1.4906

c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


185/185 ━━━━━━━━━━━━━━━━━━━━ 1715s 9s/step - accuracy: 0.3658 - loss: 1.4896 - val_accuracy: 0.5840 - val_loss: 1.1446 - learning_rate: 1.0000e-04
Epoch 2/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1540s 8s/step - accuracy: 0.6503 - loss: 1.0177 - val_accuracy: 0.6635 - val_loss: 0.9299 - learning_rate: 1.0000e-04
Epoch 3/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1497s 8s/step - accuracy: 0.7090 - loss: 0.8751 - val_accuracy: 0.7145 - val_loss: 0.8208 - learning_rate: 1.0000e-04
Epoch 4/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1480s 8s/step - accuracy: 0.7390 - loss: 0.7746 - val_accuracy: 0.7356 - val_loss: 0.7596 - learning_rate: 1.0000e-04
Epoch 5/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1487s 8s/step - accuracy: 0.7579 - loss: 0.7206 - val_accuracy: 0.7573 - val_loss: 0.7312 - learning_rate: 1.0000e-04
Epoch 6/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1548s 8s/step - accuracy: 0.7668 - loss: 0.6843 - val_accuracy: 0.7648 - val_loss: 0.6900 - learning_rate: 1.0000e-04
Epoch 7/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 1608s 9s/step - accurac